In [1]:
# Import libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import streamlit as st

In [2]:
import zipfile

with zipfile.ZipFile('Dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('path_to_extract_directory')
    print('Extraction completed.')
    new_data_dir = 'path_to_extract_directory'
    print(new_data_dir)


Extraction completed.
path_to_extract_directory


In [3]:
# Data Preprocessing and Augmentation
# Define dataset directory
new_data_dir = 'path_to_extract_directory/images.cv_jzk6llhf18tm3k0kyttxz/data'  # TODO: Update with your dataset directory

# Data Preprocessing
img_height, img_width = 224, 224  # Example for VGG16, change if needed
batch_size = 32

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(new_data_dir, 'train'),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    os.path.join(new_data_dir, 'val'),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 6225 images belonging to 11 classes.
Found 1092 images belonging to 11 classes.


In [4]:
import tensorflow as t
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Function to create a transferred model
def create_transfer_model(base_model):
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(len(train_generator.class_indices), activation='softmax'))

    base_model.trainable = False  # Freeze the base model

    return model

# Load a pre-trained model
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
transfer_learning_model = create_transfer_model(base_model)

transfer_learning_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    filepath='best_model.h5',
    monitor='val_loss',
    save_best_only=True
)

# Train transfer learning model
transfer_history = transfer_learning_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=30,
    callbacks=[early_stopping, model_checkpoint]
)

C:\Users\midun\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.4830 - loss: 1.9105

195/195 ━━━━━━━━━━━━━━━━━━━━ 1435s 7s/step - accuracy: 0.4839 - loss: 1.9068 - val_accuracy: 0.8636 - val_loss: 0.4942
Epoch 2/30
195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.8184 - loss: 0.5607

195/195 ━━━━━━━━━━━━━━━━━━━━ 1563s 8s/step - accuracy: 0.8185 - loss: 0.5602 - val_accuracy: 0.9332 - val_loss: 0.2567
Epoch 3/30
 82/195 ━━━━━━━━━━━━━━━━━━━━ 7:58 4s/step - accuracy: 0.8814 - loss: 0.3549

In [ ]:
# Evaluate the CNN Model
train_generator.reset()  # Reset generator for evaluation
cnn_eval = cnn_model.evaluate(validation_generator)
print('CNN Model Evaluation:', cnn_eval)

# Evaluate transfer learning model
transfer_learning_model.evaluate(validation_generator)

# Confusion Matrix and Classification Report
val_steps = validation_generator.samples // batch_size
predictions = cnn_model.predict(validation_generator, steps=val_steps)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

print(classification_report(true_classes, predicted_classes, target_names=class_labels))

# Confusion Matrix Plot
cm = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(10, 7))
plt.imshow(cm, interpolation='nearest')
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=45)
plt.yticks(tick_marks, class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
# Save the best model for future predictions
transfer_learning_model.save('best_transfer_learning_model.h5')

In [ ]:
# Create a Streamlit app for deployment
st.title('Fish Classification App')
st.write("Upload a fish image to classify")

uploaded_file = st.file_uploader("Choose an image...", type="jpg")

if uploaded_file is not None:
    # Load and preprocess the image
    img = tf.keras.preprocessing.image.load_img(uploaded_file, target_size=(img_height, img_width))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Perform prediction
    predictions = transfer_learning_model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    predicted_label = class_labels[predicted_class[0]]

    st.image(img, caption='Uploaded Image', use_column_width=True)
    st.write(f"Predicted Label: {predicted_label}")
    st.write(f"Confidence Score: {np.max(predictions) * 100:.2f}%")